In [31]:
import os
from Bio import SeqIO
import math
from scipy.stats import pearsonr
import datetime

In [32]:
def find_complementary(subseq:str):
    complentary_dict={'A':'U','G':'C','C':'G','U':'A'}
    comp_seq=""
    for char in subseq:
        for key, val in complentary_dict.items():
            if char == key:
                comp_seq+=val
    return comp_seq

In [33]:
'''
Function to get the complementary sequences if the other strand has a T/U
Basically U can base pair with both A and G. So, if a complementary sequence has A, 
we need all combinations of by replacing A with G in the complementary sequence.

PMID : 11256617
'''
def get_combinations(original_str, replacements_dict, results_list, current_combination):
    if not original_str:
        results_list.append(current_combination)
    else:
        first_char = original_str[0]
        remaining_str = original_str[1:]
        if first_char in replacements_dict:
            for replacement in replacements_dict[first_char]:
                get_combinations(remaining_str, replacements_dict,
                                 results_list, current_combination + replacement)
        else:
            get_combinations(remaining_str, replacements_dict,
                             results_list, current_combination + first_char)

In [34]:
def scan_complement(subseq:str,complement_seq:str):
    c=0
    for i in range (len(subseq)):
        char_search = subseq[i:i+(len(complement_seq))]
        if char_search ==  complement_seq:
            c+=1
    return c

# Snippet for checking calculations

In [35]:
'''
Snippet for checking calculations
'''
utr3_seq_dna='ATATTTCAAAATAAAGAAGAATCCACATTGCATTTGA'
utr3_seq_rna=utr3_seq_dna.replace('T','U')
print (utr3_seq_rna)
c=0
window_5mer=0
sticky_count=0
while (c < len(utr3_seq_rna)-5):
    subseq=utr3_seq_rna[c:c+5]
    window_5mer+=1
    complement_seq_rev=find_complementary(subseq)[::-1]
    print (subseq+"----"+complement_seq_rev+"\n")
    #upstream_start=c-3
    downstream_start=(c+5)+2
    #count_upstream=0
    count_downstream=0
    if 'A' not in complement_seq_rev:
        #if upstream_start > 5:
        #    upstream_seq_region=utr3_seq_rna[:upstream_start]
        #    count_upstream=scan_complement(upstream_seq_region,complement_seq_rev)
        if downstream_start <= (len(utr3_seq_rna)-5):
            downstream_seq_region=utr3_seq_rna[downstream_start:]
            count_downstream=scan_complement(downstream_seq_region,complement_seq_rev)
            print (count_downstream)
        sticky_count+=count_downstream            
    else:
        wobble_dict={'A':['A','G']}
        res=[]
        get_combinations(complement_seq_rev, wobble_dict, res, "")
        #print ('complement: '+str(res))
        #if upstream_start >= 5:
        #    count_upstream=0
        #    for i1 in range (len(res)):
        #        upstream_seq_region=utr3_seq_rna[:upstream_start]
        #        count_upstream+=scan_complement(upstream_seq_region,res[i1])
        #else:
        #    pass
        if downstream_start <= (len(utr3_seq_rna)-5):
            count_downstream=0
            for i1 in range (len(res)):
                downstream_seq_region=utr3_seq_rna[downstream_start:]
                count_downstream+=scan_complement(downstream_seq_region,res[i1])
                print (str(res[i1])+"\t"+str(count_downstream))
        
        sticky_count+=count_downstream    
    c+=5

print (window_5mer)
print (sticky_count)

AUAUUUCAAAAUAAAGAAGAAUCCACAUUGCAUUUGA
AUAUU----AAUAU

AAUAU	0
AAUGU	0
AGUAU	0
AGUGU	0
GAUAU	0
GAUGU	0
GGUAU	0
GGUGU	0
UCAAA----UUUGA

UUUGA	1
UUUGG	1
AUAAA----UUUAU

UUUAU	0
UUUGU	0
GAAGA----UCUUC

0
AUCCA----UGGAU

UGGAU	0
UGGGU	0
CAUUG----CAAUG

CAAUG	0
CAGUG	0
CGAUG	0
CGGUG	0
CAUUU----AAAUG

7
1


In [36]:
s=find_complementary('AGTTTTATTTGTTGCCCAGGGTAA')
print(s[::-1])

UUCCCUGGGCCUCU


In [37]:
wobble_dict={'A':['A','G']}
res=[]
get_combinations('AAACU',wobble_dict,res,'')
print (res)

['AAACU', 'AAGCU', 'AGACU', 'AGGCU', 'GAACU', 'GAGCU', 'GGACU', 'GGGCU']


In [38]:
s=scan_complement('AUCUACCCAAUUGGCGCGAUCACGGGG','GG')
print (s)

4


# Main working block

In [39]:
path_to_anno_file=r'/Users/basus2/Library/CloudStorage/'\
r'OneDrive-MemorialSloanKetteringCancerCenter/'\
r'Work/UTR3_RNA interaction potential/MANE_sequence_JAN2025'


#flex_infile_path=r'G:\Work\RNA interaction potential\MANE_3UTR_JAN2025'

utr3_file=os.path.join(path_to_anno_file,'mane_human_cds_exons.fasta')
#print (utr3_file)

path_to_outfile=r'/Users/basus2/Library/CloudStorage/'\
r'OneDrive-MemorialSloanKetteringCancerCenter/'\
r'Work/UTR3_RNA interaction potential/raw_output_files'
outfile=os.path.join(path_to_outfile,'CDS_stickiness_score_non_overlapping_window_5nt_only_WC_wobble_bp.txt')
print (outfile)

o1=open(outfile,'w')
line0=['ENSEMBL id','Length_CDS','Count','Stickiness_by_length','Stickiness_per_1000nts']
o1.write("\t".join(line0)+"\n")

/Users/basus2/Library/CloudStorage/OneDrive-MemorialSloanKetteringCancerCenter/Work/UTR3_RNA interaction potential/raw_output_files/CDS_stickiness_score_non_overlapping_window_5nt_only_WC_wobble_bp.txt


72

In [40]:
now = datetime.datetime.now()
print (now)
stickiness_score_list=[]
sticky_count_list=[]
utr3_length=[]
icount=0
for data in SeqIO.parse(utr3_file,"fasta"):
    header=str(data.id)
    #print (header)
    enst_id=header.strip().split("_")[2]
    utr3_seq_dna=str(data.seq)
    utr3_seq_rna=utr3_seq_dna.replace('T','U')
    #print (len(utr3_seq_rna))
    c=0
    window_5mer=0
    sticky_count=0
    while (c < len(utr3_seq_rna)-5):
        subseq=utr3_seq_rna[c:c+5]
        window_5mer+=1
        complement_seq_rev=find_complementary(subseq)[::-1]
        #print (subseq+"----"+complement_seq_rev+"\n")
        #upstream_start=c-3
        downstream_start=(c+5)+2
        #count_upstream=0
        count_downstream=0

        '''Only calculate complementary regions using watson-crick pairs'''
        '''if downstream_start <= (len(utr3_seq_rna)-5):
            downstream_seq_region = utr3_seq_rna[downstream_start:]
            count_downstream=scan_complement(downstream_seq_region,complement_seq_rev)'''
        
        '''Below if-else block is calculate complementary regions considering wobble bases
        U can pair with G along with normal watson-crick pairs'''
        
        if 'A' not in complement_seq_rev:
            if downstream_start <= (len(utr3_seq_rna)-5):
                downstream_seq_region=utr3_seq_rna[downstream_start:]
                count_downstream=scan_complement(downstream_seq_region,complement_seq_rev)

            sticky_count+=count_downstream            
        else:
            wobble_dict={'A':['A','G']}
            res=[]
            get_combinations(complement_seq_rev, wobble_dict, res, "")
            if downstream_start <= (len(utr3_seq_rna)-5):
                count_downstream=0
                for i1 in range (len(res)):
                    downstream_seq_region=utr3_seq_rna[downstream_start:]
                    count_downstream+=scan_complement(downstream_seq_region,res[i1])
            
        sticky_count+=count_downstream    
        c+=5  #Shift by 1 nucleotide,overlapping window ; NB - also run a non-overlapping ie c+=5

    try:
        stickiness_score_by_window_freq=sticky_count/window_5mer
        stickiness_score_by_utr_length=sticky_count/len(utr3_seq_rna)
        stickiness_score_per_1000_nt=(sticky_count/len(utr3_seq_rna))*1000
        #print (stickiness_score)
        #print (window_5mer)
        stickiness_score_list.append(stickiness_score_by_window_freq)
        sticky_count_list.append(sticky_count)
        utr3_length.append(len(utr3_seq_rna))
        o1.write(enst_id+"\t"+str(len(utr3_seq_rna))+"\t"+str(sticky_count)+
                 "\t"+str(stickiness_score_by_utr_length)+"\t"+str(stickiness_score_per_1000_nt)+"\n")

    except ZeroDivisionError:
        o1.write(enst_id+"\t"+str(len(utr3_seq_rna))+"\t"+"UTR5 length less than 5"+"\n")
    
    icount+=1
    if icount % 1000 == 0:
        now = datetime.datetime.now()
        print (str(now)+"\t"+str(icount))

o1.close()
pcorr=pearsonr(utr3_length,stickiness_score_list)
print ("Pearsonr : "+str(pcorr.statistic)+"with p-value of : "+str(pcorr.pvalue))

pcorr1=pearsonr(utr3_length,sticky_count_list)
print ("Pearsonr : "+str(pcorr1.statistic)+"with p-value of : "+str(pcorr1.pvalue))

2025-04-17 13:33:27.301504
2025-04-17 13:35:50.758842	1000
2025-04-17 13:39:07.370980	2000
2025-04-17 13:49:06.825397	3000
2025-04-17 13:51:56.648952	4000
2025-04-17 13:54:49.195127	5000
2025-04-17 13:57:30.253578	6000
2025-04-17 14:00:46.903967	7000
2025-04-17 14:03:21.850798	8000
2025-04-17 14:06:24.142639	9000
2025-04-17 14:08:31.394522	10000
2025-04-17 14:11:01.498844	11000
2025-04-17 14:13:11.448919	12000
2025-04-17 14:15:38.249735	13000
2025-04-17 14:19:04.216638	14000
2025-04-17 14:22:00.601895	15000
2025-04-17 14:24:10.793824	16000
2025-04-17 14:26:30.801547	17000
2025-04-17 14:28:00.698670	18000
2025-04-17 14:29:39.920976	19000
Pearsonr : 0.9655308668187743with p-value of : 0.0
Pearsonr : 0.6342343164122762with p-value of : 0.0


# DON'T KNOW...Need to find out

In [ ]:
def get_combinations(original_str, replacements_dict, results_list, current_combination):
    if not original_str:
        results_list.append(current_combination)
    else:
        first_char = original_str[0]
        remaining_str = original_str[1:]
        if first_char in replacements_dict:
            for replacement in replacements_dict[first_char]:
                get_combinations(remaining_str, replacements_dict,
                                 results_list, current_combination + replacement)
        else:
            get_combinations(remaining_str, replacements_dict,
                             results_list, current_combination + first_char)
 
 
# Example usage:
test_str = "GACGAAGAAU"
print("The original string is : " + str(test_str))
test_dict = {'A':['A','G']}
#for key in test_dict.keys():
#    if key not in test_dict[key]:
#        test_dict[key].append(key)
res = []
get_combinations(test_str, test_dict, res, "")
print("All combinations : " + str(res))
print (len(res))

# For Checking in one transcript (POC)

In [ ]:
path_to_anno_file=r'G:\Work\UTR3_RNA interaction potential\MANE_3UTR_JAN2025'

test_file=os.path.join(path_to_anno_file,'sample_utr3_exons.fasta')
header=''
utr3_seq_dna=''
for data in SeqIO.parse(test_file,'fasta'):
    header=(data.id)
    utr3_seq_dna=(data.seq)

print (len(utr3_seq_rna))
utr3_seq_rna=utr3_seq_dna.replace('T','U')


c=0
window_5mer=0
sticky_count=0
while (c < len(utr3_seq_rna)-5):
    subseq=utr3_seq_rna[c:c+5]
    window_5mer+=1
    complement_seq_rev=find_complementary(subseq)[::-1]
    #print (subseq+"----"+complement_seq_rev+"\n")
    downstream_start=(c+5)+2
    count_downstream=0
    '''Only calculate complementary regions using watson-crick pairs'''
    '''if downstream_start <= (len(utr3_seq_rna)-5):
        downstream_seq_region = utr3_seq_rna[downstream_start:]
        count_downstream=scan_complement(downstream_seq_region,complement_seq_rev)
        print (subseq+"--"+complement_seq_rev+"--"+str(count_downstream)+"\n")'''

    '''Below if-else block is calculate complementary regions considering wobble bases
        U can pair with G along with normal watson-crick pairs'''
        
    if 'A' not in complement_seq_rev:
        if downstream_start <= (len(utr3_seq_rna)-5):
            downstream_seq_region=utr3_seq_rna[downstream_start:]
            count_downstream=scan_complement(downstream_seq_region,complement_seq_rev)
            print (subseq+"--"+complement_seq_rev+"--"+str(count_downstream)+"\n")
        sticky_count+=count_downstream            
    else:
        wobble_dict={'A':['A','G']}
        res=[]
        get_combinations(complement_seq_rev, wobble_dict, res, "")
        if downstream_start <= (len(utr3_seq_rna)-5):
            count_downstream=0
            for i1 in range (len(res)):
                downstream_seq_region=utr3_seq_rna[downstream_start:]
                count_downstream+=scan_complement(downstream_seq_region,res[i1])
                if count_downstream > 0:
                    print (subseq+"--"+res[i1]+"--"+str(count_downstream)+"\n")

    sticky_count+=count_downstream    
    c+=5

print (sticky_count)